# 3. fáza projektu
## Matej Horniak, Lukáš Mikula
Dátová sada č. 44

V tomto notebooku sme zaobalili funckie z predoslej fazy (preprocessingu dat) do wrapperu. 

Tieto funkcie spustame nad jednotlivymi datami a tieto upravene data si ukladame do pomocnych csv suborov, aby sme ich mohli dalej iba citat. Taktiez spracuvavame 5. bod tohto zadania, pricom najprv vytvorime data s pouzitim inych technik nahradzovania chybajucich dat. A vysledky 5. bodu tohto zadania. Tiez zhodnocujeme vysledky nasich manualne vytvorenych stromov so stromami vytvorenymi pomocou scikit-learn.

In [14]:
import scipy.stats as stats
import pandas as pd
import numpy as np
from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.impute._iterative import IterativeImputer
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
import seaborn as sns
import dateutil.parser as parser
from sklearn.impute import SimpleImputer as Imputer, SimpleImputer
from sklearn.neighbors import NearestNeighbors as neigh
from sklearn.neighbors import KNeighborsRegressor as neigh
from sklearn.model_selection import train_test_split as split
from sklearn import metrics
from math import sqrt
import json
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from missingpy import KNNImputer, MissForest
import copy
import random

In [15]:
class DataFrameWrapper():
    def __init__(self, data, person):
        self.data = data
        self.person = person
        self.merged_data = None
        self.pipeline = None
        self.numeric1 = None
        self.numeric2 = None
        self.categoric = None
        self.preproccessors = None
        
    def getMergedData(self):
        # Return DataFrame
        return self.merged_data
    
    def setPersonOtherData(self, data, person):
        # Set new data to data and person DataFrame
        # Other variable set to None
        self.data = data
        self.person = person
        self.merged_data = None
        self.numeric1 = None
        self.numeric2 = None
        self.categoric = None
        return self
    
    def prepocessing(self):
        # Prepocessing data
        # Remove medical_info, merge other and person DataFrame
        # Remove duplicate
        # Modify data to unit format
        
        dict_array = []
        string = '{"mean_glucose": 9999999,"std_glucose": 9999999,"kurtosis_glucose":9999999,"skewness_glucose":9999999,"mean_oxygen":9999999,"std_oxygen":9999999,"kurtosis_oxygen":9999999,"skewness_oxygen":9999999}'
        self.data.medical_info.apply(lambda x: dict_array.append(json.loads(x.replace("'","\"").replace(":\"",":").replace("\",",",").replace("\"}","}") if isinstance(x,str) else string)))
        medical_datas = pd.DataFrame(dict_array)
        
        for item in medical_datas:
            self.data[item] = medical_datas[item].astype(float)
        self.data = self.data.drop(columns=['medical_info'])
        
        self.data = self.data.replace({ 9999999 : np.nan },regex=False)
        self.data = self.data.replace({'^[? ]*$' : np.nan},regex=True)
        self.person = self.person.replace({'^[? ]*$' : np.nan},regex=True)
        self.og = copy.deepcopy(self.data) 
        
        numeric = self.data.select_dtypes(include=['float64'])
        categoric = self.data.select_dtypes(include=['object'])
        numeric['name'] = categoric['name']
        
        numeric = numeric.groupby('name').aggregate('mean')
        categoric = categoric.groupby('name').aggregate('first')
        
        self.data = categoric.merge(numeric, on=['name'])
        
        self.merged_data = pd.merge(self.person, self.data, on=['name','address'], how='left')
        
        self.merged_data = self.merged_data.replace(['FALSE','F'],'f',regex=True)
        self.merged_data = self.merged_data.replace(['TRUE','T'],'t',regex=True)
        
        self.merged_data = self.merged_data.replace([' <=50K'],'<=50K',regex=True)
        self.merged_data = self.merged_data.replace([' >50K'],'>50K',regex=True)
    
    
        self.merged_data = self.merged_data.replace([' Other-relative'],'Other_relative',regex=True)
        self.merged_data = self.merged_data.replace([' Not-in-family'],'Not_in_family',regex=True)
        self.merged_data = self.merged_data.replace([' Own-child'],'Own_child',regex=True)
        
        
        for i, row in self.merged_data.iterrows():
            day = 0
            month = 0
            year = 0
            
            try:
                date = parser.parse(self.merged_data.iloc[i]['date_of_birth'].split(' ')[0])
                day = date.day
                month = date.month
                year = date.year
            except:
                day = random.randint(1,31)
                month = random.randint(1,12)
                year = 2019 - row['age']
#                 date = self.merged_data.iloc[i]['date_of_birth'].split(' ')[0]
#                 date[0] = int(date[0])
#                 date[1] = int(date[1])
#                 date[2] = int(date[2])
                
                
#                 if date[0] <= 12:
#                     month = date[0]
#                 elif date[0] <= 31:
#                     day = date[0]
#                 else:
#                     year = date[0]
                
#                 if date[1] <= 12 and month is not 0:
#                     month = date[1]
#                 elif date[1] <= 31 and day is not 0:
#                     day = date[1]
#                 else:
#                     year = date[1]
                    
#                 if date[2] <= 12 and month is not 0:
#                     month = date[2]
#                 elif date[2] <= 31 and day is not 0:
#                     day = date[2]
#                 else:
#                     year = date[2]
                
            
            
            if (year > 2019):
                year = year - 100
        
            self.merged_data.at[i,'date_of_birth'] = str(day) + "/" + str(month) + "/" + str(year)
                    
            if ((pd.isna(row['age'])) | (row['age'] > 115) | (row['age'] < 0)):
                self.merged_data.at[i,'age'] = 2019 - year
        
        return self
    
    
    def removeOutliers(self):
        # Remove outliers on floats datas
        
        ## Remove outliers from 'kurtosis_glucose'
        column = 'kurtosis_glucose'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 6) | (self.merged_data[column] < -1)
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'mean_glucose'
        column = 'mean_glucose'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 150)  
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'mean_oxygen'
        column = 'mean_oxygen'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 120)  
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'kurtosis_oxygen'
        column = 'kurtosis_oxygen'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 20)  
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'skewness_glucose'
        column = 'skewness_glucose'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 40)  
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'skewness_oxygen'
        column = 'skewness_oxygen'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 350)  
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'std_glucose'
        column = 'std_glucose'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 62) 
        self.merged_data.loc[mask ,column] = np.NaN
        
        ## Remove outliers from 'std_oxygen'
        column = 'std_oxygen'
        # sns.boxplot(self.merged_data[column])
        # self.merged_data[column].hist()
        mask = (self.merged_data[column] > 80)  
        self.merged_data.loc[mask ,column] = np.NaN
    
        return self
    
    
    def overtypeData(self):
        # Change data type of object to float64
        # From pipeline
        
        self.merged_data['age'] = self.merged_data['age'].astype(np.float64)
        self.merged_data['education-num'] = self.merged_data['education-num'].astype(np.float64)
        self.merged_data['capital-gain'] = self.merged_data['capital-gain'].astype(np.float64)
        self.merged_data['fnlwgt'] = self.merged_data['fnlwgt'].astype(np.float64)
        # self.merged_data['class'] = self.merged_data['class'].astype(np.float64)
        self.merged_data['hours-per-week'] = self.merged_data['hours-per-week'].astype(np.float64)
        self.merged_data['capital-loss'] = self.merged_data['capital-loss'].astype(np.float64)
        self.merged_data['mean_glucose'] = self.merged_data['mean_glucose'].astype(np.float64)
        self.merged_data['std_glucose'] = self.merged_data['std_glucose'].astype(np.float64)
        self.merged_data['kurtosis_glucose'] = self.merged_data['kurtosis_glucose'].astype(np.float64)
        self.merged_data['skewness_glucose'] = self.merged_data['skewness_glucose'].astype(np.float64)
        self.merged_data['mean_oxygen'] = self.merged_data['mean_oxygen'].astype(np.float64)
        self.merged_data['std_oxygen'] = self.merged_data['std_oxygen'].astype(np.float64)
        self.merged_data['kurtosis_oxygen'] = self.merged_data['kurtosis_oxygen'].astype(np.float64)
        self.merged_data['skewness_oxygen'] = self.merged_data['skewness_oxygen'].astype(np.float64)
        self.merged_data['class'] = self.person['class']
        
        return self
    
    def divideData(self):
        # Divide train data into 3 category
        # Remove age and append on the end of DataFrame
        # Remove class
        
        self.person['class'] = self.merged_data['class']
        self.merged_data = self.merged_data.drop(columns=['class'])
        age_column = copy.deepcopy(self.merged_data['age'])
        self.merged_data = self.merged_data.drop(columns=['age'])
        self.merged_data['age'] = age_column

        numeric = self.merged_data.select_dtypes(include=['float64'])
        self.numeric1 = numeric.iloc[:,:5].columns
        # print(numeric1)
        self.numeric2 = numeric.iloc[:,5:].columns
        # print(numeric2)  
        self.categoric = self.merged_data.select_dtypes(include=['object']).columns
        # print(categoric)
        return self
    
    def createPipelineSteps(self):
        # Create pipeline steps, for categoric mostfrequent strategy
        # For numeric KNN, MissForest
        
        numeric_transformer1 = Pipeline(steps=[('imputer',MissForest(missing_values=np.NaN, max_iter=10, # Nahradenie nulovych hodnot pomocou MissForest
                                                                     decreasing=False, copy=False)), 
                                              ('powerTransformer',PowerTransformer(method='yeo-johnson',
                                                                                   copy=False)) # Transformuje dat aby boli viac z Gaussoveho rozdelenia
                                               ])
        numeric_transformer2 = Pipeline(steps=[('imputer',KNNImputer(missing_values=np.NaN, n_neighbors=5, # Nahradenie nulovych hodnot pomocou KNN
                                                                      weights='distance')), 
                                               
                                               ])
        self.categoric_transormer = Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent'))]) # Nahradenie nulovych hodnot pomocou najcastejsich hodnot
                                              # ('onehot',OneHotEncoder(handle_unknown='ignore'))])
        preproccesors = ColumnTransformer(transformers=[('Cat', self.categoric_transormer, self.categoric),
                                                        ('Num1', numeric_transformer1, self.numeric1),
                                                        ('Num2', numeric_transformer2, self.numeric2)
                                                        ])
        self.preproccessors = preproccesors
        return self
    
    def createPipelineStepsMean(self):
        # Create pipeline steps, for categoric mostfrequent strategy
        # For numeric mean
        
        numeric_transformer1 = Pipeline(steps=[('imputer',SimpleImputer(strategy='mean'))])
        self.categoric_transormer = Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent'))]) # Nahradenie nulovych hodnot pomocou najcastejsich hodnot
        
        preproccesors = ColumnTransformer(transformers=[('Cat', self.categoric_transormer, self.categoric),
                                                        ('Num1', numeric_transformer1, self.numeric1),
                                                        ('Num2', numeric_transformer1, self.numeric2)
                                                        ])
        self.preproccessors = preproccesors
        return self
    
    def createPipelineStepsOnlyKNN(self):
        # Create pipeline steps, for categoric mostfrequent strategy
        # For numeric KNN
    
        numeric_transformer2 = Pipeline(steps=[('imputer',KNNImputer(missing_values=np.NaN, n_neighbors=5, # Nahradenie nulovych hodnot pomocou KNN
                                                                      weights='distance'))])
        self.categoric_transormer = Pipeline(steps=[('imputer',SimpleImputer(strategy='most_frequent'))]) # Nahradenie nulovych hodnot pomocou najcastejsich hodnot
        
        preproccesors = ColumnTransformer(transformers=[('Cat', self.categoric_transormer, self.categoric),
                                                        ('Num1', numeric_transformer2, self.numeric1),
                                                        ('Num2', numeric_transformer2, self.numeric2)
                                                        ])
        self.preproccessors = preproccesors
        return self
    
    def appendPipelineSteps(self):
        # Create Pipeline with steps
        
        if self.pipeline is None:
            print("create Pipeline")
            self.pipeline = Pipeline(steps=[('preproccesors', self.preproccessors)])
        else:
            print("append Steps")
            self.pipeline.steps.append([('preproccesors', self.preproccessors)])
        return self
    
    def popSteps(self):
        self.pipeline.steps.pop()
        return self
    
    def Fit(self):
        # Use fit in pipeline
        
        self.pipeline.fit(self.merged_data)
        # self.merged_data = pd.DataFrame(pipeline_datas,columns=self.merged_data.columns)
        return self
    
    def FitTransform(self):
        # Use fit and transform in pipeline
        
        pipeline_datas = self.pipeline.fit_transform(self.merged_data)
        self.merged_data = pd.DataFrame(pipeline_datas,columns=self.merged_data.columns)
        return self
    
    def Transform(self):
        # Use transform in pipeline
        
        pipeline_datas = self.pipeline.transform(self.merged_data)
        self.merged_data = pd.DataFrame(pipeline_datas,columns=self.merged_data.columns)
        return self
    

In [16]:
# Create new Dataset with KNN and MissForest
# Load csv files

data = pd.read_csv("../data/other_train.csv",index_col=0)
person = pd.read_csv("../data/personal_train.csv",index_col=0)


val_data = pd.read_csv("../data/other_valid.csv",index_col=0)
val_person = pd.read_csv("../data/personal_valid.csv",index_col=0)

test_data = pd.read_csv("../data/other_test.csv",index_col=0)
test_person = pd.read_csv("../data/personal_test.csv",index_col=0)

# Active train data processing

wrapper = DataFrameWrapper(data,person)
wrapper.prepocessing().removeOutliers().divideData().createPipelineSteps().appendPipelineSteps()\
                        .Fit().Transform()

# Show training data, and save into trained_datas.csv

merged_data = wrapper.overtypeData().getMergedData()
merged_data.info()
merged_data.to_csv('trained_datas.csv')

# Use pipeline to transform on valid data

wrapper.setPersonOtherData(val_data,val_person).prepocessing().removeOutliers().divideData()
wrapper.Transform().overtypeData()


# Show valid data. and save into validate_datas.csv

wrapper.getMergedData().to_csv('validate_datas.csv')
wrapper.getMergedData().info()


# Use pipeline to transform test data

wrapper.setPersonOtherData(test_data,test_person).prepocessing().removeOutliers()
age_column = copy.deepcopy(wrapper.merged_data['age'])
wrapper.merged_data = wrapper.merged_data.drop(columns=['age'])
wrapper.merged_data['age'] = age_column
wrapper.person['class'] = np.NaN
wrapper.Transform().overtypeData()

# Show test data. and save into test_datas.csv

wrapper.getMergedData().to_csv('test_datas.csv')
wrapper.getMergedData().info()

/home/matho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/matho/anaconda3/lib/python3.7/site-packages/missingpy/knnimpute.py:224: UserWarning: There are rows with more than 50.0% missing values. These rows are not included as donor neighbors.
  .format(self.row_max_missing * 100))
/home/matho/anaconda3/lib/python3.7/site-packages/missingpy/knnimpute.py:282: UserWarning: There are rows with more than 50.0% missing values. The missing features in these rows are imputed with column means.
  .format(self.row_max_missing * 100))
/home/matho/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:432: DeprecationWarning: 'warn_on_dtype' is deprecated in version 0.21 and will be removed in 0.2

create Pipeline
Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 0
Iteration: 1
Iteration: 2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933 entries, 0 to 3932
Data columns (total 28 columns):
name                3933 non-null object
address             3933 non-null object
sex                 3933 non-null object
date_of_birth       3933 non-null object
race                3933 non-null object
marital-status      3933 non-null object
occupation          3933 non-null object
pregnant            3933 non-null object
relationship        3933 non-null object
education           3933 non-null object
income              3933 non-null object
native-country      3933 non-null object
workclass           3933 non-null object
education-num       3933 non-null float64
capital-gain        3933 non-null float64
fnlwgt              3933 non-null float64
hours-per-week      3933 non-null float64
capital-loss        3933 non-null float64
kurtosis_glucose    3933 non-null float64
kurtosis_oxyge

In [17]:
test_person['date_of_birth'].value_counts()

1970-10-10             2
1961-04-27             2
69-07-01               2
1978-03-21             2
1977-12-01             2
1962-01-22             2
1967-04-21             2
1956-03-05             2
1972-08-03             2
1959-08-04             2
1984-05-20             2
1980-09-12             2
1963-09-24             2
1967-07-19             2
1965-12-11             2
1966-05-12             2
1988-04-17             2
1969-10-30             2
1960-03-10             2
1971-11-18             2
1956-03-19             2
1963-01-27             2
1956-11-16             2
1964-10-27             2
1978-03-22             2
1964-05-15             2
1966-01-17             2
1967-12-12             2
1943-10-13             1
08/01/1951             1
                      ..
1957-11-28             1
1955-10-05             1
1975-08-23             1
29/04/1966             1
1958-08-23             1
1966-05-05             1
70-10-05               1
1966-08-21 00:00:00    1
1974-08-24             1


## Vyskusame ine techniky na nahradzanie NaN hodnot, aby sme zistili ci tato cast ma vyplyv na konecne vysledky 
Vytvorili sme dve nove metody CreatePipelineStepsOnlyKNN (na nahradenie NaN hodnot
pouzije len KNN algoritmus), CreatePipelineStepsMean (na nahradenie sa nepouzije miss
forest, ale mean).

In [18]:
# Create new Dataset with only mean
# Load csv files

data = pd.read_csv("../data/other_train.csv",index_col=0)
person = pd.read_csv("../data/personal_train.csv",index_col=0)


val_data = pd.read_csv("../data/other_valid.csv",index_col=0)
val_person = pd.read_csv("../data/personal_valid.csv",index_col=0)

wrapper = DataFrameWrapper(data,person)
wrapper.prepocessing().removeOutliers().divideData().createPipelineStepsMean().appendPipelineSteps()\
                        .Fit().Transform()

# Show training data, and save into trained_datas.csv

merged_data = wrapper.overtypeData().getMergedData()
merged_data.info()
merged_data.to_csv('trained_datas_mean.csv')

# Use pipeline to transform on valid data

wrapper.setPersonOtherData(val_data,val_person).prepocessing().removeOutliers().divideData()
wrapper.Transform().overtypeData()


# Show valid data. and save into validate_datas.csv

wrapper.getMergedData().to_csv('validate_datas_mean.csv')
wrapper.getMergedData().info()

/home/matho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/matho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


create Pipeline
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933 entries, 0 to 3932
Data columns (total 28 columns):
name                3933 non-null object
address             3933 non-null object
sex                 3933 non-null object
date_of_birth       3933 non-null object
race                3933 non-null object
marital-status      3933 non-null object
occupation          3933 non-null object
pregnant            3933 non-null object
relationship        3933 non-null object
education           3933 non-null object
income              3933 non-null object
native-country      3933 non-null object
workclass           3933 non-null object
education-num       3933 non-null float64
capital-gain        3933 non-null float64
fnlwgt              3933 non-null float64
hours-per-week      3933 non-null float64
capital-loss        3933 non-null float64
kurtosis_glucose    3933 non-null float64
kurtosis_oxygen     3933 non-null float64
mean_glucose        3933 non-null float64
mean_oxy

In [19]:
# Create new Dataset with only KNN algorithm
# Load csv files

data = pd.read_csv("../data/other_train.csv",index_col=0)
person = pd.read_csv("../data/personal_train.csv",index_col=0)


val_data = pd.read_csv("../data/other_valid.csv",index_col=0)
val_person = pd.read_csv("../data/personal_valid.csv",index_col=0)

wrapper = DataFrameWrapper(data,person)
wrapper.prepocessing().removeOutliers().divideData().createPipelineStepsOnlyKNN().appendPipelineSteps()\
                        .Fit().Transform()

# Show training data, and save into trained_datas.csv

merged_data = wrapper.overtypeData().getMergedData()
merged_data.info()
merged_data.to_csv('trained_datas_KNN.csv')

# Use pipeline to transform on valid data

wrapper.setPersonOtherData(val_data,val_person).prepocessing().removeOutliers().divideData()
wrapper.Transform().overtypeData()


# Show valid data. and save into validate_datas.csv

wrapper.getMergedData().to_csv('validate_datas_KNN.csv')
wrapper.getMergedData().info()


/home/matho/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/matho/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:432: DeprecationWarning: 'warn_on_dtype' is deprecated in version 0.21 and will be removed in 0.23. Don't set `warn_on_dtype` to remove this warning.
  DeprecationWarning)
/home/matho/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:432: DeprecationWarning: 'warn_on_dtype' is deprecated in version 0.21 and will be removed in 0.23. Don't set `warn_on_dtype` to remove this warning.
  DeprecationWarning)
/home/matho/anaconda3/lib/python3.7/site-packages/missingpy/knnimpute.py:224: UserWarning: There are rows with more than 50.0% missing values. These rows

create Pipeline
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3933 entries, 0 to 3932
Data columns (total 28 columns):
name                3933 non-null object
address             3933 non-null object
sex                 3933 non-null object
date_of_birth       3933 non-null object
race                3933 non-null object
marital-status      3933 non-null object
occupation          3933 non-null object
pregnant            3933 non-null object
relationship        3933 non-null object
education           3933 non-null object
income              3933 non-null object
native-country      3933 non-null object
workclass           3933 non-null object
education-num       3933 non-null float64
capital-gain        3933 non-null float64
fnlwgt              3933 non-null float64
hours-per-week      3933 non-null float64
capital-loss        3933 non-null float64
kurtosis_glucose    3933 non-null float64
kurtosis_oxygen     3933 non-null float64
mean_glucose        3933 non-null float64
mean_oxy

## Finalne zhodnotenie jednotlivych vytvorenych stromov

Oproti natrenovanemu stromu pomocou kniznice scikit-learn, nas komplikovany strom (ktory dosiahol lepsie hodnoty ako jednoduchy) dosiahol lepsie vysledky precision a accuracy, ale recall bol o nieco horsi.

* Jednoduchy strom "nas" : precision 90.3%, recall 89.5% a accuracy 92.2%
* Komplikovany strom "nas" : precision 96.2%, recall 92% a accuracy 95.4%
* Natrenovany strom "scikit-learn" : precision 91.3%, recall 93.3% a accuracy 93.8%
* Natrenovany strom "vytunene hyperparametre" : precision 95.5%, recall 94.4% a accuracy 96.1%